In [51]:
import numpy as np
import random
import sys

data = np.loadtxt('assign1_data.txt')
#print data

x1 = data[:, 0]
print "x1:", x1

x2 = data[:, 1]
print "x2:", x2

y = data[:, 2]
#print "y:", y

z = data[:, 3]
#print "z:", z

x1: [ 0.1227  0.3914  0.7725  0.8342  0.5084  0.9983  0.6643  0.0954  0.6949
  0.6075  0.3717  0.0193  0.9314  0.2206  0.0903  0.5466  0.4331  0.5293
  0.8955  0.0701  0.0771  0.1972  0.6405  0.3203  0.103   0.5605  0.4288
  0.7101  0.6575  0.9498  0.555   0.0947  0.4293  0.31    0.7297  0.1973
  0.1571  0.2272  0.2099  0.3972  0.0033  0.6441  0.5549  0.9193  0.5341
  0.1555  0.1839  0.9949  0.7918  0.4088  0.2247  0.3901  0.4309  0.0732
  0.8077  0.0854  0.8955  0.8794  0.1094  0.0726  0.4377  0.889   0.5791
  0.9721  0.5719  0.3156  0.8065  0.4777  0.6264  0.2117  0.042   0.0916
  0.6035  0.6354  0.4102  0.695   0.2153  0.0768  0.2356  0.1591  0.7171
  0.9988  0.3936  0.0201  0.6295  0.1771  0.5888  0.5302  0.3434  0.7154
  0.3506  0.5432  0.4687  0.013   0.1311  0.697   0.3996  0.2208  0.1119
  0.4317]
x2: [ 0.299   0.6392  0.0826  0.0823  0.8025  0.7404  0.3861  0.986   0.605
  0.3618  0.761   0.7192  0.5181  0.1091  0.1085  0.2028  0.7049  0.688
  0.8477  0.6176  0.7883  0.6273  0

In [59]:
# initialize weights
w1 = random.random() * 3
w2 = random.random() * 3
b = random.random() * 3

print "w1:\t", w1
print "w2:\t", w2
print "b:\t", b

w1:	1.34409344313
w2:	1.73777136061
b:	0.569825185828


In [60]:
# online
# learning rate lr = 0.0001
lr = 0.0001 
epoch = 0
# training set size : validation set size = 3 : 1
training_set_index = 0
size = len(x1)
validation_set_index = 0
validation_set_index_step = size / 4
pre_validation_error = sys.maxint
curr_validation_error = 0
while pre_validation_error >= curr_validation_error or curr_validation_error <= 0: # early stopping when the validation error is greater than its previous one
#while True:
    
    validation_set_index += validation_set_index_step
    validation_set_index %= size
    
    x1_validation = x1[validation_set_index : validation_set_index + validation_set_index_step]
    x2_validation = x2[validation_set_index : validation_set_index + validation_set_index_step]
    y_validation = y[validation_set_index : validation_set_index + validation_set_index_step]
   
    # training
    for x1i, x2i, yi in zip(x1, x2, y):
        if x1i not in zip(x1_validation) and x2i not in zip(x2_validation) and yi not in zip(y_validation):
            w1 = w1 + lr * (yi - (x1i * w1 + x2i * w2 + b)) * x1i
            w2 = w2 + lr * (yi - (x1i * w1 + x2i * w2 + b)) * x2i
            b = b + lr * (yi - (x1i * w1 + x2i * w2 + b))

    if epoch >= 4 and epoch % 4 == 0:
        pre_validation_error = curr_validation_error
        if epoch % 100 == 0: # print only every 20 epochs
            print "epoch:\t", epoch, "\t curr_validation_error:\t", curr_validation_error
        curr_validation_error = 0
        
    # validation
    for x1i, x2i, yi in zip(x1_validation, x2_validation, y_validation):
        curr_validation_error += abs(yi - (x1i * w1 + x2i * w2 + b))
                
    epoch += 1

print "Result: w1:\t", w1, "\tw2:\t", w2, "\tb:\t", b, "\tepoch:\t", epoch
print "pre_validation_error:\t", pre_validation_error, "curr_validation_error:\t", curr_validation_error
    

epoch:	20 	 curr_validation_error:	162.991618364
epoch:	40 	 curr_validation_error:	137.566501397
epoch:	60 	 curr_validation_error:	118.871982291
epoch:	80 	 curr_validation_error:	106.297233674
epoch:	100 	 curr_validation_error:	97.2587874473
epoch:	120 	 curr_validation_error:	91.1717398784
epoch:	140 	 curr_validation_error:	87.2963928947
epoch:	160 	 curr_validation_error:	84.3754329156
epoch:	180 	 curr_validation_error:	82.2271475505
epoch:	200 	 curr_validation_error:	80.5161778498
epoch:	220 	 curr_validation_error:	79.0790732636
epoch:	240 	 curr_validation_error:	77.8857185298
epoch:	260 	 curr_validation_error:	76.8336471294
epoch:	280 	 curr_validation_error:	75.9142913663
epoch:	300 	 curr_validation_error:	75.1139905824
epoch:	320 	 curr_validation_error:	74.3621035465
epoch:	340 	 curr_validation_error:	73.6209042162
epoch:	360 	 curr_validation_error:	72.8896939707
epoch:	380 	 curr_validation_error:	72.1844260927
epoch:	400 	 curr_validation_error:	71.4860745514
epoc

epoch:	3580 	 curr_validation_error:	20.5295445959
epoch:	3600 	 curr_validation_error:	20.4485648197
epoch:	3620 	 curr_validation_error:	20.3684039065
epoch:	3640 	 curr_validation_error:	20.2904874528
epoch:	3660 	 curr_validation_error:	20.2151849082
epoch:	3680 	 curr_validation_error:	20.1406441763
epoch:	3700 	 curr_validation_error:	20.0668575441
epoch:	3720 	 curr_validation_error:	19.9952427817
epoch:	3740 	 curr_validation_error:	19.9270397646
epoch:	3760 	 curr_validation_error:	19.8595273757
epoch:	3780 	 curr_validation_error:	19.7926986169
epoch:	3800 	 curr_validation_error:	19.7265465612
epoch:	3820 	 curr_validation_error:	19.6610643517
epoch:	3840 	 curr_validation_error:	19.5962452009
epoch:	3860 	 curr_validation_error:	19.534302773
epoch:	3880 	 curr_validation_error:	19.4760577098
epoch:	3900 	 curr_validation_error:	19.4219593252
epoch:	3920 	 curr_validation_error:	19.3701845525
epoch:	3940 	 curr_validation_error:	19.3189332676
epoch:	3960 	 curr_validation_er

epoch:	7840 	 curr_validation_error:	16.3033228347
epoch:	7860 	 curr_validation_error:	16.302110266
epoch:	7880 	 curr_validation_error:	16.3009103266
epoch:	7900 	 curr_validation_error:	16.2997228854
epoch:	7920 	 curr_validation_error:	16.2985478125
epoch:	7940 	 curr_validation_error:	16.2973849796
epoch:	7960 	 curr_validation_error:	16.2962342595
epoch:	7980 	 curr_validation_error:	16.2950955264
epoch:	8000 	 curr_validation_error:	16.2939686559
epoch:	8020 	 curr_validation_error:	16.2928535247
epoch:	8040 	 curr_validation_error:	16.2917500109
epoch:	8060 	 curr_validation_error:	16.2906579939
epoch:	8080 	 curr_validation_error:	16.2895773543
epoch:	8100 	 curr_validation_error:	16.2885079738
epoch:	8120 	 curr_validation_error:	16.2874497355
epoch:	8140 	 curr_validation_error:	16.2864025238
epoch:	8160 	 curr_validation_error:	16.285366224
epoch:	8180 	 curr_validation_error:	16.2843407228
epoch:	8200 	 curr_validation_error:	16.283325908
epoch:	8220 	 curr_validation_erro

In [6]:
# minibatch


In [ ]:
# batch
